Below is TOPSIS method


In [133]:
import numpy as np
import pandas as pd
import math

In [134]:
def topsis(matrix, weights, criteria_types):
    """
    Perform TOPSIS ranking.
    
    Parameters
    ----------
    matrix : 2D array-like
        Decision matrix with shape (m, n) — m alternatives, n criteria.
    weights : 1D array-like
        Importance weights for each criterion (should sum to 1).
    criteria_types : list of str
        Each element is 'benefit' or 'cost' indicating the type of criterion.
    
    Returns
    ----------
    scores : ndarray
        Closeness coefficient for each alternative (higher = better).
    rankings : ndarray
        Indices of alternatives ranked from best to worst.
    """
    # Avoid wrong imports
    
    if sum(weights) != 1:
        raise ValueError("Weights must sum to 1.")
    if len(weights) != matrix.shape[1]:
        raise ValueError("Number of weights must match number of criteria.")
    
    # Step 1: Normalize the decision matrix
    norm_matrix = matrix / np.sqrt((matrix ** 2).sum())
    
    # Step 2: Multiply by weights
    weighted_matrix = norm_matrix * weights

    # Step 3: Determine ideal and negative-ideal solutions
    ideal_solution = []
    negative_ideal_solution = []
    for i in range(weighted_matrix.shape[1]):
        if criteria_types[i] == '+':
            ideal_solution.append(weighted_matrix[:, i].max())
            negative_ideal_solution.append(weighted_matrix[:, i].min())
        else:
            ideal_solution.append(weighted_matrix[:, i].min())
            negative_ideal_solution.append(weighted_matrix[:, i].max())
    
    ideal_solution = np.array(ideal_solution)
    negative_ideal_solution = np.array(negative_ideal_solution)

    # Step 4: Calculate distances to ideal and negative-ideal solutions
    distance_to_ideal = np.sqrt(((weighted_matrix - ideal_solution) ** 2).sum(axis=1))
    distance_to_negative_ideal = np.sqrt(((weighted_matrix - negative_ideal_solution) ** 2).sum(axis=1))

    # Step 5: Calculate performance score
    performance_score = distance_to_negative_ideal / (distance_to_ideal + distance_to_negative_ideal)

    return performance_score

In [ ]:
# from asyncio import log

def entropy(infor):
    """

    Args:
        infor (np.array): input array

    Returns:
        float: entropy value
    """
    if type(infor)!= np.array:
        infor = np.array(infor)
    return np.sum(-infor * np.log(infor + np.finfo(float).eps)) / np.log(len(infor))


# Data Standardization
def cost_indicator(matrix):
    """
    Args:
        matrix (np.array shape (n,n)): decision matrix --> axis = 0 (rows)alternatives, axis = 1 (columns)criteria
    Returns:
        np.array: cost indicator matrix
    """
    if type(matrix)!= np.array:
        matrix = np.array(matrix)
    ind = np.argmax(matrix, axis = 1)
    maxi = matrix[ind, :]
    print(maxi)
    print(np.sum(maxi - matrix, axis = 0))
    matrix = (maxi - matrix) / np.sum(maxi - matrix, axis = 0)
    return matrix

def benefit_indicator(matrix):
    """
    Args:
        matrix (np.array shape (n,n)): decision matrix --> axis = 0 (rows)alternatives, axis = 1 (columns)criteria
    Returns:
        np.array: benefit indicator matrix
    """
    if type(matrix)!= np.array:
        matrix = np.array(matrix)

    return matrix / np.sum(matrix, axis = 0)

def optimal_value_indicator(matrix, optimal_value):
    """
    Args:
        matrix (np.array shape (n,n)): decision matrix --> axis = 0 (rows)alternatives, axis = 1 (columns)criteria
    Returns:
        np.array: benefit indicator matrix
    """
    if type(matrix)!= np.array:
        matrix = np.array(matrix)
    if optimal_value is None:
        optimal_value = np.median(matrix, axis=0)
    matrix = 1 - (abs(matrix - optimal_value) / np.max(abs(matrix - optimal_value)))
    matrix = matrix / np.sum(matrix, axis = 0)
    return matrix


# Calculate Weights
def entropy_weights(matrix, indicator, optimal_value = None):
    """
    Args:
        matrix (np.array shape (n,n)): decision matrix --> axis = 0 (rows)alternatives, axis = 1 (columns)criteria
    Returns:
        np.array: entropy weights (1,n)
    """
    if type(matrix)!= np.array:
        matrix = np.array(matrix)
    # Normalization
    #matrix = matrix / matrix.sum(axis=0)
    if indicator is optimal_value_indicator:
        matrix = indicator(matrix, optimal_value)
    else:
        matrix = indicator(matrix)
    #print(matrix)
    matrix = np.where(matrix == 0, np.finfo(float).eps, matrix)
    # Entropy calculation
    n_criteria = matrix.shape[1]
    
    entropy_values = np.array(list(map(entropy, matrix.T)))
   
    #print(entropy_values)
    # Degree of diversification
    diver = 1 - entropy_values
    weights = diver / np.sum(diver)
    return weights
    

entropy_weights(np.array([[10,200,0.5],
                          [20,300,0.7],
                          [30,250,0.9]]),optimal_value_indicator)

    
    
    

array([5.23520199e-03, 9.94763180e-01, 1.61762877e-06])

In [136]:
def AHP(weights_matrix):
    """
    Perform AHP to derive weights from pairwise comparison matrix.
    
    Parameters
    ----------
    weights_matrix : 2D array-like
        Pairwise comparison matrix (n x n).
    
    Returns
    ----------
    weights : ndarray
        Derived weights for each criterion.
    """
    if type(weights_matrix)!= np.array:
        weights_matrix = np.array(weights_matrix)
    eigvals, eigvecs = np.linalg.eig(weights_matrix)
    max_index = np.argmax(eigvals)
    weights = np.real(eigvecs[:, max_index])
    weights = weights / weights.sum()
    return weights

In [137]:
AHP([[1, 3, 5],
    [1/3, 1, 2],
    [1/5, 1/2, 1]])

array([0.64832901, 0.22965079, 0.12202019])

In [138]:
# indicator = [Energy consumption, emission,waste management,water consumption, temperature]
#energy_consumption_weights = [renewable energy ratio, energy efficiency, carbon emissions,renewable energy potential]
energy_consumption_weights = [[1., 36e6/73208, 343, 2744/365, 7.518]]
carbon_emission_of_trans = [[]]
waste_management = []
water_usage = []
climate = []



**问题1：理解问题 - 环境影响指标体系**
**思路**
**建立三级指标体系：**


- 一级指标：能源、交通、废弃物、水资源、气候适应性


- 二级指标：细化的可测量指标 


- 三级指标：具体数据采集点 


**关键指标** 


1. **energy consumption**


    - 可再生能源比例（%）  


    - 场馆能源效率（kWh/座位）   


    - 电网清洁度（碳强度 gCO₂/kWh）   


    - 当地太阳能/风能潜力   


1. **carbon emission by transportation**
    - 平均观众出行距离（km）   


    - 公共交通覆盖率（%）   


    - 机场碳排放效率   


    - 电动车充电基础设施密度   


1. **waste management**
    - 回收率（%）   


    - 人均废弃物产生量（kg/人）   


    - 堆肥设施可用性   


    - 一次性塑料使用政策   


1. **water usage**
    - 水资源压力指数   


    - 雨水回收系统   


    - 节水技术应用率   


1. **climate**


    - 热岛效应强度   